In [ ]:
# Necessary Import Statements
import pandas as pd
import numpy as np


BetaData = pd.read_csv('~/ASDRP/Data/BetaData.csv', low_memory=False)
data = pd.read_csv('~/ASDRP/Data/GammaData.csv', low_memory=False)
phl = pd.read_csv('~/ASDRP/Data/PHL_Dataset.csv', low_memory=False)

print('should be 84 columns: ', len(BetaData.columns))
print('should be 92 columns: ', len(data.columns))

numberDuplicates = 0
duplicates = pd.DataFrame(columns=list(data.columns))
deDupedData = pd.DataFrame(columns=list(data.columns))
badData = pd.DataFrame(columns=list(data.columns)) # To be filled in later

# Checking if there is bad data, to find the source of the errors
areThereNulls = data['pl_pubdate'].isna().values.any()
print('Are there nulls?', areThereNulls)

howManyNulls = data['pl_pubdate'].isna().sum()
print('How many?', howManyNulls)

print('Size before: ', len(data['pl_name']))
data.dropna(how='any', subset=['pl_pubdate'], inplace=True)
print('Size after: ', len(data['pl_name']))

# Removing bad data that can't be used in any way
print('Size before removing bad data:', len(data['pl_name']))

for index, row in data.iterrows():
    date = data['pl_pubdate'][index]
    try:
        if len(date) < 8:
            data['pl_pubdate'][index] = date + '-01'
        data['pl_pubdate'][index] = pd.to_datetime(data['pl_pubdate'][index]) # This line is redundant, remove either this or line 56
    except:
#       badData.append(data.iloc[index])
        data.drop([index], inplace=True)
        data.reset_index(drop=True)

# This is what I had before doing the try, except:

    # for index, row in data.iterrows():
    #     date = data['pl_pubdate'][index]
    #     if '-00' in date or len(date) < 5 or date[4] != '-':
    # #        badData.append(data.iloc[index])
    #         data.drop([index], inplace=True)
    #         data.reset_index(drop=True)
    #     elif len(date) < 8:
    #         data['pl_pubdate'][index] = date + '-01'
    #
    # print('Number of bad data entries: (should be around 100?) ', len(badData['pl_name']))

data['pl_pubdate'] = pd.to_datetime(data['pl_pubdate'])

print('Size after removing bad data:', len(data['pl_name']))

print(data['pl_pubdate'][0:10], ' ', type(data['pl_pubdate'][0:10]))


# Sorting the data
sortedData = data.sort_values(by=['pl_name', 'pl_pubdate'], ascending= [1,0], inplace=False, na_position='last')


# Store the duplicates before removing them
duplicates = pd.DataFrame(columns=list(data.columns))
mask = sortedData.duplicated(subset='pl_name', keep='first')
df_keep = sortedData.loc[~mask]
duplicates = duplicates.append(sortedData.loc[mask])
print('Duplicates size: ', len(duplicates['pl_name']))

# Dropping the duplicates
print('Data size before:', len(sortedData['pl_name']))
deDupedData = sortedData.drop_duplicates(subset = 'pl_name', keep='first', inplace=False)
print('Data size after: ', len(deDupedData['pl_name']))

# Here I'm calculating the number of duplicates
print(len(sortedData['pl_name']) - len(deDupedData['pl_name']))
numberDuplicates = len(duplicates['pl_name'])
print('Size of duplicates: (should equal the above number) ', numberDuplicates)

print('Dataset with duplicates dropped: ', deDupedData[['pl_name', 'pl_pubdate']][0:10])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


                   pl_name pl_pubdate
0                 11 Com b 2011-08-01
1                 11 UMi b 2017-03-01
2                 14 And b 2011-08-01
3                 14 Her b 2017-03-01
4               16 Cyg B b 2017-03-01
5                 18 Del b 2011-08-01
6  1RXS J160929.1-210524 b 2015-03-01
7                 24 Boo b 2018-08-01
8                 24 Sex b 2011-01-01
9                 24 Sex c 2011-01-01
